# 聚类：寻找相似的帖子
《机器学习系统设计》第三章

## 评估帖子关联性
- 对每个帖子提取重要特征，并针对每个帖子存储为一个向量。
- 在这些向量上进行聚类计算。
- 确定每个待聚类帖子所在的簇。
- 对每个簇。获取几个与待聚类帖子不同的帖子。这样可以提升多样性。

## 预处理：用相近的公共词语个数来衡量相似性
文本处理的步骤：
- 切分文本：将原始文本转化为词袋
- 扔掉出现过于频繁，而又对检测相关贴子没有帮助的词语
- 扔掉出现频率很低，只有很小可能出现在未来帖子中的词语
- 统计剩余的词语
- 考虑整个语料集合，从词频统计中计算TF-IDF值

词袋模型缺点：
- 不涵盖词语之间的关联关系。"Car hits wall"和"Wall hits car"
- 没法正确捕捉否定关系。"I will eat ice cream"和"I will not eat ice cream"从特征向量来看非常相似。解决方法使用unigrams(单个词语),bigrams(成对的词语)或者trigrams(一行中的三个词语)
- 对于拼写错误的词语会处理失败。

### 导入语料

In [8]:
import os
import sys

DATA_DIR = os.path.join('./', "data")

if not os.path.exists(DATA_DIR):
    print("Uh, we were expecting a data directory, which contains the toy data")
    sys.exit(1)

CHART_DIR = os.path.join('./', "charts")
if not os.path.exists(CHART_DIR):
    os.mkdir(CHART_DIR)

TOY_DIR = os.path.join(DATA_DIR, "toy")
posts = [open(os.path.join(TOY_DIR, f)).read() for f in os.listdir(TOY_DIR)]
for p in posts:
    print p.strip()

This is a toy post about machine learning. Actually, it contains not much interesting stuff.
Imaging databases provide storage capabilities.
Most imaging databases save images permanently.
Imaging databases store data.
Imaging databases store data. Imaging databases store data. Imaging databases store data.


### TF-IDF
当某个词语经常出现在一些特定帖子中，而在其他地方很少出现，会给该词语较高的权值。

In [10]:
import scipy as sp


def tfidf(t, d, D):  # term, doc of term, doc set
    tf = float(d.count(t)) / sum(d.count(w) for w in set(d))
    idf = sp.log(float(len(D)) / (len([doc for doc in D if t in doc])))
    return tf * idf


a, abb, abc = ["a"], ["a", "b", "b"], ["a", "b", "c"]
D = [a, abb, abc]

print(tfidf("a", a, D))
print(tfidf("b", abb, D))
print(tfidf("a", abc, D))
print(tfidf("b", abc, D))
print(tfidf("c", abc, D))

0.0
0.270310072072
0.0
0.135155036036
0.366204096223


### 完整流程

In [9]:
import os
import sys
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


import nltk.stem
english_stemmer = nltk.stem.SnowballStemmer('english')  # 词干处理


class StemmedCountVectorizer(CountVectorizer):

    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))


class StemmedTfidfVectorizer(TfidfVectorizer):

    def build_analyzer(self):
        analyzer = super(StemmedTfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))


def train(vectorizer):
    X_train = vectorizer.fit_transform(posts)  # 向量化

    num_samples, num_features = X_train.shape
    print("#samples: %d, #features: %d" % (num_samples, num_features))

    new_post_vec = vectorizer.transform([new_post])
    print(new_post_vec, type(new_post_vec))
    print(new_post_vec.toarray())
    print(vectorizer.get_feature_names())
    return X_train, new_post_vec


def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())  # 向量的2范数


def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())  # 向量归一化
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())

    delta = v1_normalized - v2_normalized

    return sp.linalg.norm(delta.toarray())


def evaluate(model, new_post_vec):
    best_dist = sys.maxsize
    best_i = None
    num_samples, num_features = model.shape

    for i in range(0, num_samples):
        post = posts[i]
        if post == new_post:
            continue
        post_vec = model.getrow(i)
        d = dist(post_vec, new_post_vec)

        print("=== Post %i with dist=%.2f: %s" % (i, d, post))

        if d < best_dist:
            best_dist = d
            best_i = i

    print("Best post is %i with dist=%.2f" % (best_i, best_dist))

new_post = "imaging databases"

# vectorizer = CountVectorizer(min_df=1, stop_words='english', preprocessor=stemmer)
stemmedCountVectorizer = StemmedCountVectorizer(min_df=1, stop_words='english')
stemmedTfidfVectorizer = StemmedTfidfVectorizer(min_df=1, stop_words='english', decode_error='ignore')
dist = dist_norm

print '===stemmedCountVectorizer==='
model, new_post_vec = train(stemmedCountVectorizer)
evaluate(model, new_post_vec)

print '===stemmedTfidfVectorizer==='
model, new_post_vec = train(stemmedTfidfVectorizer)
evaluate(model, new_post_vec)

===stemmedCountVectorizer===
#samples: 5, #features: 17
(<1x17 sparse matrix of type '<type 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>, <class 'scipy.sparse.csr.csr_matrix'>)
[[0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0]]
[u'actual', u'capabl', u'contain', u'data', u'databas', u'imag', u'interest', u'learn', u'machin', u'perman', u'post', u'provid', u'save', u'storag', u'store', u'stuff', u'toy']
=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.
=== Post 2 with dist=0.63: Most imaging databases save images permanently.

=== Post 3 with dist=0.77: Imaging databases store data.
=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 2 with dist=0.63
===stemmedTfidfVectorizer===
#samples: 5, #features: 17
(<1x17 sparse matrix of type '<type 'numpy.float

## 聚类
20newsgroup数据集是机器学习中的一个标准数据集。来自不同新闻组。我们选取一部分作为训练集。

In [1]:
import scipy as sp
import sklearn.datasets
all_data = sklearn.datasets.fetch_20newsgroups(subset="all")
print("Number of total posts: %i" % len(all_data.filenames))

groups = [
    'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware',
    'comp.sys.mac.hardware', 'comp.windows.x', 'sci.space']
train_data = sklearn.datasets.fetch_20newsgroups(subset="train",
                                                 categories=groups)
print("Number of training posts in tech groups:", len(train_data.filenames))
# Number of training posts in tech groups: 3529

labels = train_data.target
sp.unique(labels).shape

Number of total posts: 18846
('Number of training posts in tech groups:', 3529)


(6,)

训练集属于六个组，所以我们定义kmeans的簇为6

In [2]:
num_clusters = 6  # sp.unique(labels).shape[0]

对帖子做向量化预处理

In [3]:
import nltk.stem
english_stemmer = nltk.stem.SnowballStemmer('english')

from sklearn.feature_extraction.text import TfidfVectorizer


class StemmedTfidfVectorizer(TfidfVectorizer):

    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

vectorizer = StemmedTfidfVectorizer(min_df=10, max_df=0.5,
                                    stop_words='english', decode_error='ignore'
                                    )

vectorized = vectorizer.fit_transform(train_data.data)
num_samples, num_features = vectorized.shape
print("#samples: %d, #features: %d" % (num_samples, num_features))  # row 3529， column 4712

#samples: 3529, #features: 4712


对帖子做聚类

In [4]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=num_clusters, n_init=1, verbose=1, random_state=3)
clustered = km.fit(vectorized)

print("km.labels_=%s" % km.labels_)
# km.labels_=[[1 1 1 ..., 3 3 0]] 6个cluster,由id=0,1,...,5代表

print("km.labels_.shape=%s" % km.labels_.shape)
# km.labels_.shape=3529 这3529个点分别属于哪个cluster

from sklearn import metrics
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand Index: %0.3f" % metrics.adjusted_rand_score(labels, km.labels_))
print("Adjusted Mutual Information: %0.3f" % metrics.adjusted_mutual_info_score(labels, km.labels_))
print(("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(vectorized, labels, sample_size=1000)))

Initialization complete
Iteration  0, inertia 6511.359
Iteration  1, inertia 3400.718
Iteration  2, inertia 3385.624
Iteration  3, inertia 3378.951
Iteration  4, inertia 3375.098
Iteration  5, inertia 3373.139
Iteration  6, inertia 3371.640
Iteration  7, inertia 3370.683
Iteration  8, inertia 3369.839
Iteration  9, inertia 3369.309
Iteration 10, inertia 3369.086
Iteration 11, inertia 3368.998
Iteration 12, inertia 3368.950
Iteration 13, inertia 3368.927
Converged at iteration 13
km.labels_=[1 1 1 ..., 3 3 0]
km.labels_.shape=3529
Homogeneity: 0.211
Completeness: 0.262
V-measure: 0.233
Adjusted Rand Index: 0.109
Adjusted Mutual Information: 0.209
Silhouette Coefficient: 0.006


新的帖子做向量化，并给出聚类结果

In [5]:
new_post = \
    """Disk drive problems. Hi, I have a problem with my hard disk.
After 1 year it is working only sporadically now.
I tried to format it, but now it doesn't boot any more.
Any ideas? Thanks.
"""
new_post_vec = vectorizer.transform([new_post])
new_post_label = km.predict(new_post_vec)[0]  # 5 意味着和cluster5相似
new_post_label

5

既然有了聚类信息，我们不必要用new_post_vec和所有帖子的向量进行比较，相反，我们只需要专注于同一个簇中的帖子。

In [6]:
similar_indices = (km.labels_ == new_post_label).nonzero()[0]

括号中比较操作得到一个布尔型数组，nonzero将这个数组转化为一个更小的数组，它包含True元素的索引。使用similar_indices构建一个帖子列表，以及它们的相似度分值。

In [7]:
similar = []
for i in similar_indices:
    dist = sp.linalg.norm((new_post_vec - vectorized[i]).toarray())
    similar.append((dist, train_data.data[i]))

similar = sorted(similar)
print("Count similar: %i" % len(similar))

show_at_1 = similar[0]
show_at_2 = similar[int(len(similar) / 10)]
show_at_3 = similar[int(len(similar) / 2)]

print("=== #1 ===")
print(show_at_1)
print()

print("=== #2 ===")
print(show_at_2)
print()

print("=== #3 ===")
print(show_at_3)

Count similar: 226
=== #1 ===
(1.0378441731334074, u"From: Thomas Dachsel <GERTHD@mvs.sas.com>\nSubject: BOOT PROBLEM with IDE controller\nNntp-Posting-Host: sdcmvs.mvs.sas.com\nOrganization: SAS Institute Inc.\nLines: 25\n\nHi,\nI've got a Multi I/O card (IDE controller + serial/parallel\ninterface) and two floppy drives (5 1/4, 3 1/2) and a\nQuantum ProDrive 80AT connected to it.\nI was able to format the hard disk, but I could not boot from\nit. I can boot from drive A: (which disk drive does not matter)\nbut if I remove the disk from drive A and press the reset switch,\nthe LED of drive A: continues to glow, and the hard disk is\nnot accessed at all.\nI guess this must be a problem of either the Multi I/o card\nor floppy disk drive settings (jumper configuration?)\nDoes someone have any hint what could be the reason for it.\nPlease reply by email to GERTHD@MVS.SAS.COM\nThanks,\nThomas\n+-------------------------------------------------------------------+\n| Thomas Dachsel          